In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os
import string
import io
import json

In [2]:
file = open("data.txt", "r", encoding = "utf8")
data = file.read()
data = data.split()
new_data = []
for i in data:
    i = i.replace('\n', '').replace('\r', '').replace('\ufeff', '')
    i = i.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    i = i.lower()
    new_data.append(i)
new_data

['one',
 'morning ',
 'when',
 'gregor',
 'samsa',
 'woke',
 'from',
 'troubled',
 'dreams ',
 'he',
 'found',
 'himself',
 'transformed',
 'in',
 'his',
 'bed',
 'into',
 'a',
 'horrible',
 'vermin ',
 'he',
 'lay',
 'on',
 'his',
 'armour like',
 'back ',
 'and',
 'if',
 'he',
 'lifted',
 'his',
 'head',
 'a',
 'little',
 'he',
 'could',
 'see',
 'his',
 'brown',
 'belly ',
 'slightly',
 'domed',
 'and',
 'divided',
 'by',
 'arches',
 'into',
 'stiff',
 'sections ',
 'the',
 'bedding',
 'was',
 'hardly',
 'able',
 'to',
 'cover',
 'it',
 'and',
 'seemed',
 'ready',
 'to',
 'slide',
 'off',
 'any',
 'moment ',
 'his',
 'many',
 'legs ',
 'pitifully',
 'thin',
 'compared',
 'with',
 'the',
 'size',
 'of',
 'the',
 'rest',
 'of',
 'him ',
 'waved',
 'about',
 'helplessly',
 'as',
 'he',
 'looked ',
 ' what s',
 'happened',
 'to',
 'me  ',
 'he',
 'thought ',
 'it',
 'wasn t',
 'a',
 'dream ',
 'his',
 'room ',
 'a',
 'proper',
 'human',
 'room',
 'although',
 'a',
 'little',
 'too',
 's

In [3]:
input_array = []
for i in new_data:
    if i not in input_array:
        
        input_array.append(i)
data = " ".join(input_array)
data

'one morning  when gregor samsa woke from troubled dreams  he found himself transformed in his bed into a horrible vermin  lay on armour like back  and if lifted head little could see brown belly  slightly domed divided by arches stiff sections  the bedding was hardly able to cover it seemed ready slide off any moment  many legs  pitifully thin compared with size of rest him  waved about helplessly as looked   what s happened me   thought  wasn t dream  room  proper human room although too small  peacefully between its four familiar walls  collection textile samples spread out table   travelling salesman above there hung picture that had recently cut an illustrated magazine housed nice  gilded frame  showed lady fitted fur hat boa who sat upright  raising heavy muff covered whole her lower arm towards viewer  then turned look window at dull weather  drops rain be heard hitting pane  which made him feel quite sad   how i sleep bit longer forget all this nonsense   but something unable d

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

sequence_data = tokenizer.texts_to_sequences([data])[0]
len(sequence_data)

3488

In [6]:
print(tokenizer.word_index)

{'s': 1, 't': 2, 'you': 3, 'i': 4, 'll': 5, 'he': 6, 'it': 7, 'that': 8, 'd': 9, 'mother': 10, 'we': 11, 'gregor': 12, 'well': 13, 'now': 14, 'they': 15, 're': 16, 'then': 17, 'god': 18, 'clock': 19, 'no': 20, 'grete': 21, 'one': 22, 'samsa': 23, 'in': 24, 'like': 25, 'him': 26, 'what': 27, 'there': 28, 'look': 29, 'all': 30, 'this': 31, 'something': 32, 'so': 33, 'oh': 34, 'can': 35, 'let': 36, 'yes': 37, 'father': 38, 'today': 39, 'help': 40, 'she': 41, 'everyone': 42, 'bed': 43, 'on': 44, 'and': 45, 'if': 46, 'see': 47, 'to': 48, 'moment': 49, 'happened': 50, 'me': 51, 'room': 52, 'out': 53, 'be': 54, 'but': 55, 'do': 56, 'right': 57, 'present': 58, 'where': 59, 'pain': 60, 'before': 61, 'is': 62, 've': 63, 'more': 64, 'time': 65, 'know': 66, 'go': 67, 'other': 68, 'boss': 69, 'would': 70, 'everything': 71, 'seven': 72, 'o': 73, 'furniture': 74, 'did': 75, 'come': 76, 'round': 77, 'doctor': 78, 'ever': 79, 'wrong': 80, 'door': 81, 'somewhere': 82, 'away': 83, 'family': 84, 'while': 

In [7]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

2573


In [8]:
sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
print(sequences)

The Length of sequences are:  3487
[[  22  114]
 [ 114  669]
 [ 669   12]
 ...
 [2571  116]
 [ 116  515]
 [ 515 2572]]


In [9]:
X = []
y = []

for i in sequences:
    X.append(i[0])
    y.append(i[1])
    
X = np.array(X)
y = np.array(y)

In [10]:
print("The Data is: ", X[:6])
print("The responses are: ", y[:6])

The Data is:  [ 22 114 669  12  23 670]
The responses are:  [114 669  12  23 670 115]


In [11]:
y = to_categorical(y, num_classes=vocab_size)
y.shape

(3487, 2573)

In [12]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [13]:
checkpoint_path = "training/cp-{epoch:04d}.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True)

In [14]:
model.compile(loss="categorical_crossentropy", optimizer='adam',metrics = ['accuracy'])

In [ ]:
model.fit(X, y, epochs=100, batch_size=64, callbacks=[cp_callback])

Epoch 1/100
55/55 [==============================] - 8s 81ms/step - loss: 7.8558 - accuracy: 0.0094

Epoch 00001: saving model to training/cp-0001.ckpt
Epoch 2/100
55/55 [==============================] - 5s 85ms/step - loss: 7.8495 - accuracy: 0.0114

Epoch 00002: saving model to training/cp-0002.ckpt
Epoch 3/100
55/55 [==============================] - 4s 80ms/step - loss: 7.8408 - accuracy: 0.0081

Epoch 00003: saving model to training/cp-0003.ckpt
Epoch 4/100
55/55 [==============================] - 4s 78ms/step - loss: 7.6920 - accuracy: 0.0132

Epoch 00004: saving model to training/cp-0004.ckpt
Epoch 5/100
55/55 [==============================] - 5s 83ms/step - loss: 7.4427 - accuracy: 0.0080

Epoch 00005: saving model to training/cp-0005.ckpt
Epoch 6/100
55/55 [==============================] - 4s 79ms/step - loss: 7.2510 - accuracy: 0.0106

Epoch 00006: saving model to training/cp-0006.ckpt
Epoch 7/100
55/55 [==============================] - 4s 81ms/step - loss: 7.1115 - accur

55/55 [==============================] - 4s 79ms/step - loss: 2.2697 - accuracy: 0.3109

Epoch 00054: saving model to training/cp-0054.ckpt
Epoch 55/100
55/55 [==============================] - 4s 78ms/step - loss: 2.1698 - accuracy: 0.3175

Epoch 00055: saving model to training/cp-0055.ckpt
Epoch 56/100
55/55 [==============================] - 4s 77ms/step - loss: 2.0933 - accuracy: 0.3362

Epoch 00056: saving model to training/cp-0056.ckpt
Epoch 57/100
55/55 [==============================] - 4s 78ms/step - loss: 2.0928 - accuracy: 0.3442

Epoch 00057: saving model to training/cp-0057.ckpt
Epoch 58/100
55/55 [==============================] - 4s 81ms/step - loss: 2.0788 - accuracy: 0.3454

Epoch 00058: saving model to training/cp-0058.ckpt
Epoch 59/100
55/55 [==============================] - 4s 78ms/step - loss: 1.9880 - accuracy: 0.3670

Epoch 00059: saving model to training/cp-0059.ckpt
Epoch 60/100
55/55 [==============================] - 4s 79ms/step - loss: 1.9495 - accuracy: 0

In [ ]:
model.save('my_model.h5')

In [ ]:
model = tf.keras.models.load_model('my_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("my_model.tflite", "wb").write(tflite_model)

In [ ]:
word_index = []
for i in tokenizer.word_index:
    a = {i :tokenizer.word_index[i]}
    word_index.append(a)

with io.open('word_index.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(word_index, ensure_ascii=False))

In [ ]:
index_word = []
for i in tokenizer.index_word:
    a = {i :tokenizer.index_word[i]}
    index_word.append(a)

with io.open('index_word.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(index_word, ensure_ascii=False))